In [ ]:
import pandas as pd
import numpy as np

import numpy as np
from mujoco_playground import registry

import sys
sys.path.append("..")
from algorithms.utils.wrapper_gym import GymWrapper

In [3]:
!export XLA_PYTHON_CLIENT_PREALLOCATE=false 

In [ ]:
path_model = [
    {'env': 'H1JoystickGaitTracking', "model": "random"},
    {'env': 'H1InplaceGaitTracking', "model": "random"},
    {'env': 'Go1JoystickRoughTerrain', "model": "random"}, 
    {'env': 'Go1JoystickFlatTerrain', "model": "random"},
    {'env': 'Go1Handstand', "model": "random"}, 
    {'env': 'Go1Getup', "model": "random"}, 
    {'env': 'Go1Footstand', "model": "random"},
    {'env': 'G1JoystickRoughTerrain', "model": "random"}, 
    {'env': 'G1JoystickFlatTerrain', "model": "random"},
]

In [ ]:
for p in path_model:
    print("-"*100)
    print(f"ENV: {p['env']}")
    print("-"*100)
    print()
    
    env = registry.load(p["env"])
    env_cfg = registry.get_default_config(p["env"])
    randomizer = registry.get_domain_randomizer(p["env"])

    render_trajectory = []

    def render_callback(_, state):
        render_trajectory.append(state)

    env_wrapped = GymWrapper(
        env,
        num_actors=1,
        seed=1,
        episode_length=env_cfg.episode_length,
        action_repeat=1,
        render_callback=render_callback,
        randomization_fn=randomizer,
        device="cuda",
    )

    state_dim = env_wrapped.observation_space.shape[0] 
    action_dim = env_wrapped.action_space.shape[0]
    max_action = 1.0

    # ------------- RANDOM EVALUATION
    def eval_random(env, n_episodes):
        episode_rewards = []
        for _ in range(n_episodes):
            _, _ = env.reset()
            done = False
            episode_reward = 0.0
            while not done:
                action = np.random.randn(action_dim)
                _, reward, done, _, _ = env.step(action)
                episode_reward += reward
            episode_rewards.append(episode_reward)

        return np.asarray(episode_rewards)
    
    episode_rewards = eval_random(env_wrapped, 10)
    p["episode_rewards_mean"] = episode_rewards.mean()
    p["episode_rewards_std"] = episode_rewards.std()

In [ ]:
pd.DataFrame.from_dict(path_model)

In [12]:
pd.DataFrame.from_dict(path_model).to_csv("results_random.csv", index=False)